# <center> *SQL Queries - Covid 19* <center/>

In [1]:
from database.db_config import current_db as db

In [2]:
#  Alter the headers types, before querying.
db.execute_ddl('ALTER TABLE "All continents updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer, ALTER COLUMN "Date" SET DATA TYPE date USING to_date(cast("Date" as TEXT), \'yyyy-mm-dd\')')
db.execute_ddl('ALTER TABLE "All countries updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer,ALTER COLUMN "Date" SET DATA TYPE date USING to_date(cast("Date" as TEXT), \'yyyy-mm-dd\')')


DDL executed successfully.
DDL executed successfully.


In [3]:
countries_minMax = db.sql_query('SELECT "Country", "TotalCases" FROM "All countries updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All countries updated" LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All countries updated"LIMIT 1)))')
print(countries_minMax)

+------------+------------+
|  Country   | TotalCases |
+------------+------------+
|    USA     |  28453526  |
|  Vanuatu   |     1      |
| Micronesia |     1      |
+------------+------------+


In [4]:
# TotalCases Min and Max by a continent.
continents_minMax = db.sql_query('SELECT "Continent", "TotalCases" FROM "All continents updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All continents updated" WHERE "Continent" <>  \'World\' LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All continents updated" WHERE "Continent" <>  \'World\'LIMIT 1)))')
print(continents_minMax)

+---------------+------------+
|   Continent   | TotalCases |
+---------------+------------+
| North America |  32598096  |
|    Oceania    |   50671    |
+---------------+------------+


In [5]:
# Top 3 countries with the highest number of cases by Continents.
top3_totalCases = db.sql_query('SELECT * '
                                'FROM (SELECT "Country","Continent","TotalCases",RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "TotalCases_Rank","ActiveCases","SeriousCritical"'
                                'FROM (SELECT "Country_id","Continent_id"'
                                        'FROM "All Countries") AS t1 JOIN  (SELECT * '
                                                                            'FROM "All Continents") AS t2 USING("Continent_id")'
                                        'JOIN  (SELECT "Country","Country_id","TotalCases","ActiveCases","SeriousCritical"'
                                                'FROM "All countries updated" ) AS t3 USING("Country_id")) AS "t1"'
                                'WHERE "TotalCases_Rank" <=3 ')

print(top3_totalCases)

+------------------+---------------+------------+-----------------+-------------+-----------------+
|     Country      |   Continent   | TotalCases | TotalCases_Rank | ActiveCases | SeriousCritical |
+------------------+---------------+------------+-----------------+-------------+-----------------+
|   South Africa   |     Africa    |  1496439   |        1        |    48132    |       546       |
|     Morocco      |     Africa    |   479579   |        2        |     9596    |       263       |
|     Tunisia      |     Africa    |   225116   |        3        |    32044    |       288       |
|      India       |      Asia     |  10949546  |        1        |    138802   |       8944      |
|      Turkey      |      Asia     |  2609359   |        2        |    84788    |       1193      |
|       Iran       |      Asia     |  1542076   |        3        |    165280   |       3684      |
|      Russia      |     Europe    |  4112151   |        1        |    388123   |       2300      |


In [6]:
# Countries that are located in Oceania continent.
oceania_countries = db.sql_query('SELECT "Country" '
                                 'FROM "All Countries" '
                                 'WHERE "Continent_id" IN (SELECT "Continent_id" '
                                                            'FROM "All Continents" '
                                 '                           WHERE "Continent" = \'Oceania\')')
print(oceania_countries)

+------------------+
|     Country      |
+------------------+
|    Australia     |
|   New Zealand    |
| French Polynesia |
| Papua New Guinea |
|       Fiji       |
|  New Caledonia   |
+------------------+


In [7]:
# The count of countries in each continent.
countries_perContinent = db.sql_query('SELECT "Continent","Countires_count" FROM (SELECT * FROM "All Continents") AS t1 '
                                      'JOIN (SELECT "Continent_id", COUNT("Country") AS "Countires_count" '
                                      'FROM "All Countries" G'
                                      'ROUP BY "Continent_id") AS t2 '
                                      'USING("Continent_id") '
                                      'ORDER BY "Countires_count" DESC')
print(countries_perContinent)

+---------------+-----------------+
|   Continent   | Countires_count |
+---------------+-----------------+
|     Africa    |        57       |
|      Asia     |        49       |
|     Europe    |        48       |
| North America |        39       |
| South America |        14       |
|    Oceania    |        6        |
+---------------+-----------------+


In [8]:
# The country with the highest "TotalCases" in each continent.
country_high_perContinent = db.sql_query('SELECT "Country","Continent","TotalCases" '
                                         'FROM (SELECT "Country","Continent","TotalCases" ,RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "rank" '
                                                'FROM (SELECT "Country","Country_id","TotalCases" '
                                                       'FROM "All countries updated" ) AS t1  '
                                                       'JOIN (SELECT "Country_id", "Continent_id" FROM "All Countries") AS t2 '
                                                        'USING("Country_id") '
                                                        'JOIN (SELECT "Continent_id","Continent" FROM "All Continents") AS t3 '
                                                        'USING("Continent_id")) AS t4 '
                                         'WHERE "rank" =1;')
print(country_high_perContinent)

+--------------+---------------+------------+
|   Country    |   Continent   | TotalCases |
+--------------+---------------+------------+
| South Africa |     Africa    |  1496439   |
|    India     |      Asia     |  10949546  |
|    Russia    |     Europe    |  4112151   |
|     USA      | North America |  28453526  |
|  Australia   |    Oceania    |   28911    |
|    Brazil    | South America |  9979276   |
+--------------+---------------+------------+


In [9]:
# Top 5 countries with active cases.
top5_countries_active = db.sql_query('SELECT "Country","ActiveCases" '
                                     'FROM "All countries updated" '
                                     'WHERE "ActiveCases" IS NOT NULL '
                                     'ORDER BY "ActiveCases" DESC LIMIT 5;')
print(top5_countries_active)

+---------+-------------+
| Country | ActiveCases |
+---------+-------------+
|   USA   |   9354485   |
|  France |   3185288   |
|    UK   |   1669549   |
|  Brazil |    786648   |
| Belgium |    668525   |
+---------+-------------+


In [10]:
# The date that israel had the highest value of new parm.
israel_max_dates = db.sql_query('SELECT "max_NewCases_date","max_NewDeaths_date","max_NewRecovered_date"'
                                    'FROM (SELECT "Date" AS "max_NewDeaths_date" ,"Country_id" FROM "Israel" WHERE "NewDeaths" = (SELECT MAX("NewDeaths") FROM "Israel")) AS "i1"'
                                    'FULL OUTER JOIN'
                                    '(SELECT "Date" AS "max_NewRecovered_date" ,"Country_id" FROM "Israel" WHERE "NewRecovered" = (Select MAX("NewRecovered") FROM "Israel")) AS "i2"'
                                        'USING("Country_id") '
                                    'FULL OUTER JOIN '
                                    '(SELECT "Date" AS "max_NewCases_date" ,"Country_id" FROM "Israel" WHERE "NewCases" = (Select MAX("NewCases") FROM "Israel")) AS "i3"'
                                        'USING("Country_id")')
print(israel_max_dates)

+-------------------+--------------------+-----------------------+
| max_NewCases_date | max_NewDeaths_date | max_NewRecovered_date |
+-------------------+--------------------+-----------------------+
|     2021-01-28    |     2021-01-28     |       2021-02-06      |
+-------------------+--------------------+-----------------------+


In [11]:
# The date that israel had the lowest value of new parm.
israel_min_dates = db.sql_query('SELECT MIN("min_NewCases_date") AS "min_NewCases_date" ,MIN("min_NewDeaths_date") AS "min_NewDeaths_date",MIN("min_NewRecovered_date") AS "min_NewRecovered_date"'
                                    'FROM (SELECT "Date" AS "min_NewDeaths_date" ,"Country_id" FROM "Israel" WHERE "NewDeaths" = (SELECT MIN("NewDeaths") FROM "Israel")) AS "i1"'
                                    'FULL OUTER JOIN'
                                    '(SELECT "Date" AS "min_NewRecovered_date" ,"Country_id" FROM "Israel" WHERE "NewRecovered" = (Select MIN("NewRecovered") FROM "Israel")) AS "i2"'
                                        'USING("Country_id") '
                                    'FULL OUTER JOIN '
                                    '(SELECT "Date" AS "min_NewCases_date" ,"Country_id" FROM "Israel" WHERE "NewCases" = (Select MIN("NewCases") FROM "Israel")) AS "i3"'
                                        'USING("Country_id")')
print(israel_min_dates)

+-------------------+--------------------+-----------------------+
| min_NewCases_date | min_NewDeaths_date | min_NewRecovered_date |
+-------------------+--------------------+-----------------------+
|     2020-08-29    |     2020-08-29     |       2020-08-29      |
+-------------------+--------------------+-----------------------+


In [12]:
# Ranking each continent by its active cases.
continent_active_rank = db.sql_query('SELECT "Continent","ActiveCases", RANK() OVER(ORDER BY "ActiveCases" DESC) '
                                     'FROM "All continents updated" '
                                     'WHERE "Continent" <> \'World\'')
print(continent_active_rank)

+---------------+-------------+------+
|   Continent   | ActiveCases | rank |
+---------------+-------------+------+
|     Europe    |   10166776  |  1   |
| North America |   9883918   |  2   |
| South America |   1163911   |  3   |
|      Asia     |   1063212   |  4   |
|     Africa    |    363100   |  5   |
|    Oceania    |    15371    |  6   |
+---------------+-------------+------+


In [13]:
israel_minMax_dates = db.sql_query('SELECT MIN("Date") as "Earliest Date",MAX("Date") as "Latest Date" '
                                   'FROM "Israel"')
print(israel_minMax_dates)

+---------------+-------------+
| Earliest Date | Latest Date |
+---------------+-------------+
|   2020-08-02  |  2021-02-17 |
+---------------+-------------+
